In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
%cd ../hots
import tonic, torch, os, pickle, sys
from tqdm import tqdm
from network import network
from layer import mlrlayer
from timesurface import timesurface
from utils import apply_jitter, get_loader, get_sliced_loader, make_histogram_classification, HOTS_Dataset, fit_mlr, predict_mlr, score_classif_events, plotjitter, printfig
import matplotlib.pyplot as plt
import numpy as np

print(f'Tonic version installed -> {tonic.__version__}')

print(f'Number of GPU devices available: {torch.cuda.device_count()}')
for N_gpu in range(torch.cuda.device_count()):
    print(f'GPU {N_gpu+1} named {torch.cuda.get_device_name(N_gpu)}')
    
import zipfile as z
from torchvision.datasets.vision import VisionDataset


class DVSGesture_dataset(VisionDataset):
    """DVSGesture <http://research.ibm.com/dvsgesture/> dataset, either reduced or foveated.
    arguments:
        repertory: root repertory where the different processed datasets are stored 
        type_data: must be either test (to load test data), reduce (to load reduced data) or fovea (to load foveated data)
        spatial_divider: spatial dividing factor (all data is divided by 4, no need to change default)
        structural_divider: structural dividing factor, to keep only a certain percentage of the processed data 
                    (if want to charge the whole processed data, set structural_divider to 100)
        method: spacial downscaling method 
                    (if type_data not 'test', roi_method must be either 'funnelling', 'eventcount', 'linear' or 'cubic')
        roi_method: ROI data to use during foveation 
                    if type_data is 'fovea' and roi is maintained to default None, then use data foveated on whole input data ; 
                    else, use data foveated on downscaled data using a certain method)
    """

    classes = [
        "hand_clapping",
        "right_hand_wave",
        "left_hand_wave",
        "right_arm_clockwise",
        "right_arm_counter_clockwise",
        "left_arm_clockwise",
        "left_arm_counter_clockwise",
        "arm_roll",
        "air_drums",
        "air_guitar",
        "other_gestures",
    ]

    sensor_size = (128, 128)
    ordering = "xypt"
    dtype = np.dtype([("x", np.int16), ("y", np.int16), ("p", bool), ("t", np.int64)])

    def __init__(
        self, repertory, type_data, spatial_divider=4, structural_divider=100, method=None, roi_method=None, 
    transform = tonic.transforms.NumpyAsType(int)):
        super(DVSGesture_dataset, self).__init__(repertory)
        assert type_data in ['test', 'reduce', 'fovea'], "Wrong 'type_data' argument"

        if type_data == 'test':
            self.zip_name = type_data
            self.folder_name = 'Test data/'
        
        elif type_data == 'reduce':
            assert method in ['funnelling', 'eventcount', 'linear', 'cubic'], "Wrong 'method' argument"
            self.zip_name = 'reduced_data_'+method+'_div'+str(spatial_divider)
            self.folder_name = 'Reduced data/Method - '+method+'/'
        
        else : 
            assert method in ['funnelling', 'eventcount', 'linear', 'cubic'], "Wrong 'method' argument"
            assert roi_method in ['funnelling', 'eventcount', 'linear', 'cubic', None], "Wrong 'roi_method' argument"
            
            if roi_method == None:
                roi_method = 'no reduc'
            self.folder_name = 'Foveated data/ROI data - '+roi_method+'/Method - '+method+'/'
            
            if roi_method == 'no reduc':
                roi_method = 'none'
            self.zip_name = 'foveated_data_'+method+'_div'+str(spatial_divider)+'_ROI'+roi_method

        if structural_divider != 100:
            assert structural_divider in [5,10,20,40,60,80], "Wrong 'structural_divider' argument"
            self.zip_name += '_'+str(structural_divider)+'%'

        self.location_on_system = repertory
        self.data = []
        self.samples = []
        self.targets = []
        #self.transform = transform

        file_path = os.path.join(self.location_on_system, self.folder_name, self.zip_name)
        
        if not os.path.exists(file_path) and os.path.exists(file_path+'.zip'):
            
            print('Extracting into '+file_path+'...')
            with z.ZipFile(file_path+'.zip', 'r') as zip_dir :
                zip_dir.extractall(os.path.join(self.location_on_system, self.folder_name))
            print('Extraction done')

        if os.path.exists(file_path):
    
            for path, dirs, files in os.walk(file_path):
                dirs.sort()
                for file in files:
                    if file.endswith("npy"):
                        self.samples.append(path + "/" + file)
                        self.targets.append(int(file[:-4]))

        else: 
            print('Error: The folder '+file_path+' does not exist')
            sys.exit()

    def __getitem__(self, index):
        events = np.load(self.samples[index])
        events[:, 3] *= 1000  # convert from ms to us
        target = self.targets[index]
        events = np.lib.recfunctions.unstructured_to_structured(events, self.dtype)
        if self.transform is not None:
            events = self.transform(events, self.sensor_size, self.ordering)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return events, target

    def __len__(self):
        return len(self.samples)

/home/antoine/homhots/hotsline/hots
Tonic version installed -> 1.2.1
Number of GPU devices available: 1
GPU 1 named GeForce RTX 2080 Ti


In [22]:
%ls

__init__.py   __pycache__/  layer.pyc   timesurface.py
__init__.pyc  layer.py      network.py  utils.py


In [23]:
testset.folder_name

'Reduced data/Method - cubic/'

In [5]:
from tonic import SlicedDataset
from tonic.slicers import SliceByTime

dataset = tonic.datasets.NMNIST(save_to="../../Data", train=False)

slicing_time_window = 1000  # microseconds
slicer = SliceByTime(time_window=slicing_time_window)
sliced_dataset = SlicedDataset(
    dataset, slicer=slicer, metadata_path="./metadata/nmnist"
)

AttributeError: module 'tonic.datasets' has no attribute 'PokerDVS'

In [11]:
kfold_test = None
kfold_clust = 10
ts_batch_size = 1000

dataset_name = 'gesture'
slicing_time_window = 1e6

type_transform = tonic.transforms.NumpyAsType(int)
#trainset = tonic.datasets.DVSGesture(save_to='../../Data/', train=True, transform=type_transform)

repertory = '../../Data/DVSGesture_reduced'
type_data = 'reduce'
method = 'cubic'
roi_method = None
shuffle = False
num_workers = 0

testset = DVSGesture_dataset(repertory, type_data, spatial_divider=4, structural_divider=100, method=method, roi_method=roi_method)

metadata_path = testset.location_on_system+'/'+testset.folder_name+'metadata'
slicer = tonic.slicers.SliceAtTimePoints(start_tw = [0], end_tw = [slicing_time_window])
sliced_dataset = tonic.SlicedDataset(testset, slicer = slicer, transform = type_transform, metadata_path = metadata_path)
loader = torch.utils.data.DataLoader(sliced_dataset, shuffle=shuffle, num_workers = num_workers)

Metadata written to ../../Data/DVSGesture_reduced/Reduced data/Method - cubic/metadata/slice_metadata.h5.


In [12]:
events, target = next(iter(loader))

In [13]:
print(events)

tensor([[[      2,      98,       1, 4952831],
         [      0,      97,       1, 4952831],
         [      0,      96,       1, 4952831],
         ...,
         [     37,     106,       1,  998590],
         [     37,     107,       1,  998590],
         [     36,     106,       1,  998590]]])


In [ ]:
print